In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import tensorflow as tf
import glob
import json
import numpy as np
import random
import os
import tensorflow_io as tfio

In [2]:
def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_it_all()

In [3]:
class CFG:
    left_ROWS_per_frame = 21
    sequence_length = 20
    batch_size = 32

labels  = json.load(open('sign_to_prediction_index_map.json','r'))
complete_df = pd.read_csv('train.csv')
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(complete_df, test_size=0.1)

In [4]:
def build_loader(with_labels=True):
    def load_video(video_path):
        #print('herer')
        video_df = tfio.IODataset.from_parquet(video_path)
        #video_df = pd.read_parquet(video_path, engine='pyarrow')
        #video_df.fillna(0,inplace=True)
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y','z']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y','z']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        return [left_hand_array, right_hand_array]
    
    def load_video_with_labels(path, label):
        return load_video(path), labels[label]
    
    return load_video_with_labels if with_labels else load_video

In [7]:
class CustomData(tf.keras.utils.Sequence):
    def __init__(self,df,num_frames=20,batch_size=8,shuffle=True,\
                 labels_path='sign_to_prediction_index_map.json'):
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_frames = num_frames
        self.labels  = json.load(open('sign_to_prediction_index_map.json','r'))
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self,index):
        batches = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        left_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,2))
        right_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,2))
        labels = []
        for i,row_val in enumerate(batches):
            row = self.df.iloc[row_val]
            left_hand,right_hand = self.load_video(row['path'])
            left_hand_input[i,:] = left_hand
            right_hand_input[i,:] = right_hand
            labels.append(self.labels[row['sign']])
        return [left_hand_input,right_hand_input],np.asarray(labels)
            
    def load_video(self,video_path):
        video_df = pd.read_parquet(video_path, engine='pyarrow')
        video_df.dropna(inplace=True)
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,2)
        if len(left_values)!=0:
            left_values[:,:,0] = (left_values[:,:,0]- np.min(left_values[:,:,0]))/(left_values[:,:,0].max()- left_values[:,:,0].min())
            left_values[:,:,1] = (left_values[:,:,1]- np.min(left_values[:,:,1]))/(left_values[:,:,1].max()- left_values[:,:,1].min())
            left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        else:
            left_hand_array =  tf.zeros(shape=(CFG.sequence_length, CFG.left_ROWS_per_frame,2),dtype=tf.float32)
        
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,2)
        if len(right_values) != 0:
            right_values[:,:,0] = (right_values[:,:,0]- np.min(right_values[:,:,0]))/(right_values[:,:,0].max()- right_values[:,:,0].min())
            right_values[:,:,1] = (right_values[:,:,1]- np.min(right_values[:,:,1]))/(right_values[:,:,1].max()- right_values[:,:,1].min())
            right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        else:
            right_hand_array =  tf.zeros(shape=(CFG.sequence_length, CFG.left_ROWS_per_frame,2),dtype=tf.float32)
        return left_hand_array, right_hand_array
    
    def __len__(self):
        return len(self.df)//self.batch_size

In [5]:
extended_df = pd.read_csv('extended_train.csv')

In [9]:
extended_df.head()

,path,participant_id,sequence_id,sign,start_frame,end_frame,total_frames,face_count,face_nan_count,pose_count,...,left_hand_count,left_hand_nan_count,right_hand_count,right_hand_nan_count,x_min,x_max,y_min,y_max,z_min,z_max
0,/kaggle/input/asl-signs/train_landmark_files/2...,26734,1000035562,blow,20,42,23,10764,0,759,...,483,483,483,252,-0.031811,1.294350,-2.147826,-0.224151,-2.782624,1.910815
1,/kaggle/input/asl-signs/train_landmark_files/2...,28656,1000106739,wait,29,39,11,5148,0,363,...,231,231,231,189,-0.017062,1.015931,-2.222732,-0.329273,-2.543970,1.627621
2,/kaggle/input/asl-signs/train_landmark_files/1...,16069,100015657,cloud,103,207,105,49140,0,3465,...,2205,1617,2205,2205,-0.042923,1.197836,-2.591290,-0.248094,-2.838325,1.587503
3,/kaggle/input/asl-signs/train_landmark_files/2...,25571,1000210073,bird,17,28,12,5616,0,396,...,252,252,252,0,-0.129268,1.156573,-2.294936,-0.310272,-3.018237,2.196296
4,/kaggle/input/asl-signs/train_landmark_files/6...,62590,1000240708,owie,22,39,18,8424,0,594,...,378,378,378,0,0.072425,1.165405,-2.264609,-0.405282,-2.970331,1.360011


In [ ]:
for index,row in extended_df.iterrows():
    left_hand_array, right_hand_array = train_datagen.load_video(row['path'])
    

In [8]:
train_datagen = CustomData(train_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)
test_datagen = CustomData(test_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)

In [5]:
def conv1d_lstm_block(inputs, filters):
    vector = tf.keras.layers.ConvLSTM1D(filters=32, kernel_size=3,return_sequences=True)(inputs)
    vector = tf.keras.layers.Dropout(0.2)(vector)
    vector = tf.keras.layers.BatchNormalization(axis=-1)(vector)
    vector = tf.keras.layers.ConvLSTM1D(filters=64, kernel_size=3,return_sequences=True)(vector)
    vector = tf.keras.layers.Dropout(0.2)(vector)
    vector = tf.keras.layers.BatchNormalization(axis=-1)(vector)
    vector = tf.keras.layers.ConvLSTM1D(filters=64, kernel_size=3)(vector)
    vector = tf.keras.layers.Dropout(0.2)(vector)
    return vector

def get_model():
    input1 = tf.keras.Input((CFG.sequence_length, CFG.left_ROWS_per_frame, 2), dtype=tf.float32)
    input2 = tf.keras.Input((CFG.sequence_length, CFG.left_ROWS_per_frame, 2), dtype=tf.float32)
    left_hand_vector = conv1d_lstm_block(input1, [64])
    right_hand_vector = conv1d_lstm_block(input2, [64])
    vector = tf.keras.layers.Concatenate(axis=1)([left_hand_vector, right_hand_vector])
    vector = tf.keras.layers.Flatten()(vector)
    output = tf.keras.layers.Dense(250, activation="softmax")(vector)
    model = tf.keras.Model(inputs=[input1,input2], outputs=output)
    model.compile(tf.keras.optimizers.Adam(0.000333), "sparse_categorical_crossentropy", metrics="accuracy")
    return model

In [6]:
model = get_model()

ValueError: Exception encountered when calling layer "conv_lstm1d_2" (type ConvLSTM1D).

Negative dimension size caused by subtracting 9 from 5 for '{{node conv_lstm1d_2/convolution}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv_lstm1d_2/convolution/ExpandDims, conv_lstm1d_2/convolution/ExpandDims_1)' with input shapes: [?,1,5,64], [1,9,64,64].

Call arguments received by layer "conv_lstm1d_2" (type ConvLSTM1D):
  • inputs=tf.Tensor(shape=(None, 20, 5, 64), dtype=float32)
  • mask=None
  • training=None
  • initial_state=None

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 21, 2)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 20, 21, 2)]  0           []                               
                                                                                                  
 conv_lstm1d (ConvLSTM1D)       (None, 20, 19, 32)   13184       ['input_1[0][0]']                
                                                                                                  
 conv_lstm1d_3 (ConvLSTM1D)     (None, 20, 19, 32)   13184       ['input_2[0][0]']                
                                                                                              

In [10]:
file_name = "models/030923_00_26.h5"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        file_name, 
        save_best_only=True, 
        monitor="val_accuracy",
        mode="max",
        verbose = 1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,mode='max',verbose=1,
                              patience=3, min_lr=0.000001)
]
model.fit(train_datagen,validation_data=test_datagen,\
          epochs=30, callbacks=callbacks)
model = tf.keras.models.load_model(file_name)

Epoch 1/30
2361/2361 [==============================] - ETA: 0s - loss: 3.5007 - accuracy: 0.2448
Epoch 1: val_accuracy improved from -inf to 0.38692, saving model to models/030923_00_26.h5
2361/2361 [==============================] - 1125s 472ms/step - loss: 3.5007 - accuracy: 0.2448 - val_loss: 2.6387 - val_accuracy: 0.3869 - lr: 3.3300e-04
Epoch 2/30
2361/2361 [==============================] - ETA: 0s - loss: 2.4072 - accuracy: 0.4283
Epoch 2: val_accuracy improved from 0.38692 to 0.46944, saving model to models/030923_00_26.h5
2361/2361 [==============================] - 1105s 468ms/step - loss: 2.4072 - accuracy: 0.4283 - val_loss: 2.2377 - val_accuracy: 0.4694 - lr: 3.3300e-04
Epoch 3/30
2361/2361 [==============================] - ETA: 0s - loss: 2.0382 - accuracy: 0.5058
Epoch 3: val_accuracy improved from 0.46944 to 0.51404, saving model to models/030923_00_26.h5
2361/2361 [==============================] - 1107s 469ms/step - loss: 2.0382 - accuracy: 0.5058 - val_loss: 2.0206

Epoch 25/30
2361/2361 [==============================] - ETA: 0s - loss: 0.5595 - accuracy: 0.8325
Epoch 25: val_accuracy did not improve from 0.63437

Epoch 25: ReduceLROnPlateau reducing learning rate to 3.330000035930425e-05.
2361/2361 [==============================] - 1105s 468ms/step - loss: 0.5595 - accuracy: 0.8325 - val_loss: 1.7206 - val_accuracy: 0.6333 - lr: 3.3300e-04
Epoch 26/30
2361/2361 [==============================] - ETA: 0s - loss: 0.4033 - accuracy: 0.8845
Epoch 26: val_accuracy improved from 0.63437 to 0.65095, saving model to models/030923_00_26.h5
2361/2361 [==============================] - 1106s 468ms/step - loss: 0.4033 - accuracy: 0.8845 - val_loss: 1.6577 - val_accuracy: 0.6510 - lr: 3.3300e-05
Epoch 27/30
2361/2361 [==============================] - ETA: 0s - loss: 0.3587 - accuracy: 0.8986
Epoch 27: val_accuracy improved from 0.65095 to 0.65445, saving model to models/030923_00_26.h5
2361/2361 [==============================] - 1106s 468ms/step - loss: 0

In [11]:
model.save('models/030523_00_09.h5')